# Crypto Trading Bot with Live Updates and Trade Indicators

In [1]:
import ccxt
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pygame
import threading

buy_signal= "C:/Users/sbrec/OneDrive/Documents/Sound Recordings/Buy.mp3"
sell_signal= "C:/Users/sbrec/OneDrive/Documents/Sound Recordings/Sell.mp3"

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## One Market Dashboard

In [4]:
# Initialize Pygame for audio playback
pygame.mixer.init()
buy_sound_file = buy_signal  # Replace with your file path
sell_sound_file = sell_signal  # Replace with your file path

# Initialize Coinbase Exchange
exchange = ccxt.coinbase()
symbol = 'DOGE/USD'  # Dogecoin pair

# Supported timeframes
timeframes = {
    '6m': '1d',  # Last 6 months = Daily data
    '3m': '1d',  # Last 3 months = Daily data
    '1m': '1h',  # Last 1 month = Hourly data
    '5d': '1h',  # Last 5 days = Hourly data
    '1d': '15m',  # Last 1 day = 15-minute data
    '4h': '5m',  # Last 4 hours = 5-minute data
    '1h': '1m'   # Last 1 hour = Minute data
}

# Fetch historical data
def fetch_data(timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Calculate RSI
def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    return df

# Calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20):
    df['middle_band'] = df['close'].rolling(window=period).mean()
    std_dev = df['close'].rolling(window=period).std()
    df['upper_band'] = df['middle_band'] + (2 * std_dev)
    df['lower_band'] = df['middle_band'] - (2 * std_dev)
    return df

# Calculate MACD
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['ema_fast'] = df['close'].ewm(span=fast_period, adjust=False).mean()
    df['ema_slow'] = df['close'].ewm(span=slow_period, adjust=False).mean()
    df['macd'] = df['ema_fast'] - df['ema_slow']
    df['macd_signal'] = df['macd'].ewm(span=signal_period, adjust=False).mean()
    return df

# Calculate SMA
def calculate_sma(df, period=50):
    df['sma'] = df['close'].rolling(window=period).mean()
    return df

# Add indicators
def add_indicators(df):
    df = calculate_rsi(df)
    df = calculate_bollinger_bands(df)
    df = calculate_macd(df)
    df = calculate_sma(df)
    return df

# Generate buy/sell signals
def generate_signals(df):
    buy_signals = df[(df['rsi'] < 30)]
    sell_signals = df[(df['rsi'] > 70)]
    return buy_signals, sell_signals

# Play sound alerts
def play_sound(signal):
    def sound_thread():
        if signal == 'buy':
            pygame.mixer.music.load(buy_sound_file)
        elif signal == 'sell':
            pygame.mixer.music.load(sell_sound_file)
        pygame.mixer.music.play()
    threading.Thread(target=sound_thread).start()

# Create Dash app
app = dash.Dash(__name__)
app.title = "DOGE Trading Bot"

# App Layout
app.layout = html.Div(
    style={'backgroundColor': '#121212', 'color': 'white', 'padding': '10px'},
    children=[
        html.H1("DOGE Trading Bot Dashboard", style={'textAlign': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='timeframe-selector',
                options=[{'label': k, 'value': v} for k, v in timeframes.items()],
                value='1d',  # Default timeframe
                style={'width': '50%', 'margin': 'auto', 'color': 'black'}
            )
        ]),
        dcc.Graph(id='trading-chart'),
        html.Div(id='alerts', style={'textAlign': 'center', 'fontSize': '20px', 'marginTop': '20px'}),
        dcc.Interval(id='interval-update', interval=30*1000, n_intervals=0)  # Update every 30 seconds
    ]
)

# Callback for updating the chart
@app.callback(
    [Output('trading-chart', 'figure'), Output('alerts', 'children')],
    [Input('timeframe-selector', 'value'), Input('interval-update', 'n_intervals')]
)
def update_chart(timeframe, n_intervals):
    df = fetch_data(timeframe)
    df = add_indicators(df)
    buy_signals, sell_signals = generate_signals(df)
    
    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df['timestamp'], open=df['open'], high=df['high'],
        low=df['low'], close=df['close'],
        name='Candlesticks', increasing_line_color='#00cc96', decreasing_line_color='#ff6361'
    ))

    # Add Bollinger Bands
    fig.add_trace(go.Scatter(
        x=df['timestamp'], y=df['upper_band'],
        line=dict(color='rgba(255, 211, 0, 0.5)', width=1), name='Upper Band'
    ))
    fig.add_trace(go.Scatter(
        x=df['timestamp'], y=df['lower_band'],
        line=dict(color='rgba(255, 211, 0, 0.5)', width=1), name='Lower Band'
    ))

    # Add SMA
    fig.add_trace(go.Scatter(
        x=df['timestamp'], y=df['sma'],
        line=dict(color='blue', width=2), name='SMA'
    ))

    # Add Buy/Sell signals
    fig.add_trace(go.Scatter(
        x=buy_signals['timestamp'], y=buy_signals['close'],
        mode='markers', marker=dict(color='green', size=10, symbol='triangle-up'),
        name='Buy Signal'
    ))
    fig.add_trace(go.Scatter(
        x=sell_signals['timestamp'], y=sell_signals['close'],
        mode='markers', marker=dict(color='red', size=10, symbol='triangle-down'),
        name='Sell Signal'
    ))

    # Dual Y-axes
    fig.update_layout(
        template='plotly_dark',
        title=f"DOGE/USD - {timeframe} Timeframe",
        xaxis_title="Timestamp",
        yaxis=dict(title="Price (USD)", side="left"),
        yaxis2=dict(title="Price (USD)", side="right", overlaying="y", showgrid=False),
        height=600
    )

    # Alerts
    latest_signal = ''
    if len(buy_signals) > 0 and buy_signals['timestamp'].iloc[-1] == df['timestamp'].iloc[-1]:
        latest_signal = 'Buy Signal Triggered!'
        play_sound('buy')
    elif len(sell_signals) > 0 and sell_signals['timestamp'].iloc[-1] == df['timestamp'].iloc[-1]:
        latest_signal = 'Sell Signal Triggered!'
        play_sound('sell')

    return fig, latest_signal

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

## Multiple Markets Dashboard

In [2]:

# Initialize Pygame for audio playback
pygame.mixer.init()
buy_sound_file = buy_signal 
sell_sound_file = sell_signal  

# Initialize Coinbase Exchange
exchange = ccxt.coinbase()

# Supported coins and their symbols
symbols = {
    "DOGE": "DOGE/USD",
    "ADA": "ADA/USD",
    "XRP": "XRP/USD"
}

# Supported timeframes
timeframes = {
    '6m': '1d',  # Last 6 months = Daily data
    '3m': '1d',  # Last 3 months = Daily data
    '1m': '1h',  # Last 1 month = Hourly data
    '5d': '1h',  # Last 5 days = Hourly data
    '1d': '15m',  # Last 1 day = 15-minute data
    '4h': '5m',  # Last 4 hours = 5-minute data
    '1h': '1m'   # Last 1 hour = Minute data
}

# Fetch historical data
def fetch_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Calculate RSI
def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    return df

# Calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20):
    df['middle_band'] = df['close'].rolling(window=period).mean()
    std_dev = df['close'].rolling(window=period).std()
    df['upper_band'] = df['middle_band'] + (2 * std_dev)
    df['lower_band'] = df['middle_band'] - (2 * std_dev)
    return df

# Calculate SMA
def calculate_sma(df, period=50):
    df['sma'] = df['close'].rolling(window=period).mean()
    return df

# Add indicators
def add_indicators(df):
    df = calculate_rsi(df)
    df = calculate_bollinger_bands(df)
    df = calculate_sma(df)
    return df

# Generate buy/sell signals
def generate_signals(df):
    buy_signals = df[(df['rsi'] < 30)]
    sell_signals = df[(df['rsi'] > 70)]
    return buy_signals, sell_signals

# Play sound alerts
def play_sound(signal):
    def sound_thread():
        if signal == 'buy':
            pygame.mixer.music.load(buy_sound_file)
        elif signal == 'sell':
            pygame.mixer.music.load(sell_sound_file)
        pygame.mixer.music.play()
    threading.Thread(target=sound_thread).start()

# Create Dash app
app = dash.Dash(__name__)
app.title = "Crypto Trading Dashboard"

# App Layout
app.layout = html.Div(
    style={'backgroundColor': '#121212', 'color': 'white', 'padding': '10px'},
    children=[
        html.H1("Crypto Trading Dashboard", style={'textAlign': 'center'}),
        html.Div([
            html.Div([
                html.H2("DOGE/USD"),
                dcc.Dropdown(
                    id='doge-timeframe-selector',
                    options=[{'label': k, 'value': v} for k, v in timeframes.items()],
                    value='1d',  # Default timeframe
                    style={'width': '80%', 'margin': 'auto', 'color': 'black'}
                ),
                dcc.Graph(id='doge-chart')
            ], style={'marginBottom': '50px'}),
            html.Div([
                html.H2("ADA/USD"),
                dcc.Dropdown(
                    id='ada-timeframe-selector',
                    options=[{'label': k, 'value': v} for k, v in timeframes.items()],
                    value='1d',
                    style={'width': '80%', 'margin': 'auto', 'color': 'black'}
                ),
                dcc.Graph(id='ada-chart')
            ], style={'marginBottom': '50px'}),
            html.Div([
                html.H2("XRP/USD"),
                dcc.Dropdown(
                    id='xrp-timeframe-selector',
                    options=[{'label': k, 'value': v} for k, v in timeframes.items()],
                    value='1d',
                    style={'width': '80%', 'margin': 'auto', 'color': 'black'}
                ),
                dcc.Graph(id='xrp-chart')
            ])
        ]),
        dcc.Interval(id='interval-update', interval=30*1000, n_intervals=0)  # Update every 30 seconds
    ]
)

# Callback for updating charts
@app.callback(
    [Output('doge-chart', 'figure'), Output('ada-chart', 'figure'), Output('xrp-chart', 'figure')],
    [
        Input('doge-timeframe-selector', 'value'),
        Input('ada-timeframe-selector', 'value'),
        Input('xrp-timeframe-selector', 'value'),
        Input('interval-update', 'n_intervals')
    ]
)
def update_charts(doge_timeframe, ada_timeframe, xrp_timeframe, n_intervals):
    charts = []
    for symbol, timeframe in zip(['DOGE', 'ADA', 'XRP'], [doge_timeframe, ada_timeframe, xrp_timeframe]):
        df = fetch_data(symbols[symbol], timeframe)
        df = add_indicators(df)
        buy_signals, sell_signals = generate_signals(df)
        
        # Create the chart
        fig = go.Figure()
        fig.add_trace(go.Candlestick(
            x=df['timestamp'], open=df['open'], high=df['high'],
            low=df['low'], close=df['close'],
            name='Candlesticks', increasing_line_color='#00cc96', decreasing_line_color='#ff6361'
        ))
        fig.add_trace(go.Scatter(
            x=df['timestamp'], y=df['sma'],
            line=dict(color='blue', width=2), name='SMA'
        ))
        fig.add_trace(go.Scatter(
            x=buy_signals['timestamp'], y=buy_signals['close'],
            mode='markers', marker=dict(color='green', size=10, symbol='triangle-up'),
            name='Buy Signal'
        ))
        fig.add_trace(go.Scatter(
            x=sell_signals['timestamp'], y=sell_signals['close'],
            mode='markers', marker=dict(color='red', size=10, symbol='triangle-down'),
            name='Sell Signal'
        ))
        fig.update_layout(
            template='plotly_dark',
            title=f"{symbol}/USD - {timeframe} Timeframe",
            xaxis_title="Timestamp",
            yaxis_title="Price (USD)",
            height=500
        )
        charts.append(fig)

    return charts[0], charts[1], charts[2]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)